In [1]:
# Linear Regression

In [35]:
import datetime as dt

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

import sys

sys.path.append("..")
from fetching_from_local_db.enums import AssetClass, Index, StrikeSpread
from fetching_from_local_db.fetch_from_db import (
    _fetch_batch,
    fetch_data,
    fetch_spot_data,
)
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [ ]:
def get_expiry(f_today):
    days_to_thursday = (3 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry

In [62]:
async def get_expiry(f_today, symbol):
    if symbol == 'bnf':
        if (f_today <= dt.date(2024, 1, 25)) and (f_today >= dt.date(2024, 1, 18)):
            f_expiry = dt.date(2024, 1, 25)
        elif (f_today <= dt.date(2024, 1, 31)) and (f_today >= dt.date(2024, 1, 26)):
            f_expiry = dt.date(2024, 1, 31)
        elif (f_today <= dt.date(2024, 2, 22)) and (f_today >= dt.date(2024, 2, 29)):
            f_expiry = dt.date(2024, 2, 29)
        elif (f_today <= dt.date(2024, 3, 25)) and (f_today >= dt.date(2024, 3, 27)):
            f_expiry = dt.date(2024, 2, 27)
        elif f_today < dt.date(2023, 9, 1):
            days_to_thursday = (3 - f_today.weekday()) % 7
            nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
            f_expiry = nearest_thursday
            if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
                f_expiry = nearest_thursday - dt.timedelta(days=1)
        elif f_today >= dt.date(2023, 9, 1):
            if f_today.day < 24:
                days_to_wednesday = (2 - f_today.weekday()) % 7
                nearest_wednesday = f_today + dt.timedelta(days=days_to_wednesday)
                f_expiry = nearest_wednesday
                if nse.valid_days(
                    start_date=nearest_wednesday, end_date=nearest_wednesday
                ).empty:
                    f_expiry = nearest_wednesday - dt.timedelta(days=1)
            else:
                days_to_thursday = (3 - f_today.weekday()) % 7
                nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
                f_expiry = nearest_thursday
                if nse.valid_days(
                    start_date=nearest_thursday, end_date=nearest_thursday
                ).empty:
                    f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry
    elif symbol == 'nifty':
        days_to_thursday = (3 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif symbol == 'fnf':
        days_to_thursday = (1 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif symbol == 'midcp':
        days_to_thursday = (0 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    else:
        return 0

In [ ]:
def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"

In [37]:
bnf_pandas = pd.read_csv("../data/bnf.csv")

In [254]:
symbol = 'bnf'
# symbol = 'nifty'
# symbol = 'fnf'
# symbol = 'midcp'

if symbol == 'bnf':
    LEVERAGE_ = 5
    LOT_SIZE_ = 15
    SLIPPAGE_ = 0.01
    SPREAD_ = 100
elif symbol == 'nifty' or 'fnf':
    LEVERAGE_ = 7
    LOT_SIZE_ = 25
    SLIPPAGE_ = 0.01
    SPREAD_ = 50
elif symbol == 'midcp':
    LEVERAGE_ = 10
    LOT_SIZE_ = 50
    SLIPPAGE_ = 0.01
    SPREAD_ = 25

PORTFOLIO = 10000000

In [39]:
bnf_pandas.tail()

,datetime,open,high,low,close,volume
687621,2024-06-07 15:25:00,49800.4500,49812.7500,49792.7000,49806.2000,0
687622,2024-06-07 15:26:00,49807.2500,49811.7000,49798.7500,49807.5000,0
687623,2024-06-07 15:27:00,49810.8500,49823.8000,49797.6000,49817.3000,0
687624,2024-06-07 15:28:00,49816.2500,49827.6000,49805.8000,49816.0000,0
687625,2024-06-07 15:29:00,49820.3000,49825.0000,49805.2500,49806.6000,0


In [40]:
# If Stocks Data ...
bnf_pandas["datetime"] = pd.to_datetime(bnf_pandas["datetime"])
bnf_pandas["datetime"] = bnf_pandas["datetime"].dt.tz_localize(None)
bnf_pandas = bnf_pandas[bnf_pandas["datetime"].dt.year >= 2017]
# bnf_pandas.drop(columns=["datetime"], inplace=True)
# bnf_pandas

In [41]:
bnf = pl.DataFrame(bnf_pandas)
print(type(bnf))
# bnf

<class 'polars.dataframe.frame.DataFrame'>


In [42]:
# # For crude oil Data
# bnf1 = pd.DataFrame(bnf_pandas)
# bnf1 = bnf1.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])
# bnf1['datetime'] = pd.to_datetime(bnf1['datetime'])
# bnf1['index'] = bnf1['datetime']
# bnf1.rename(columns={'o': 'open', 'h': 'high', 'l': 'low', 'c': 'close'}, inplace=True)
# bnf = pl.DataFrame(bnf1)
# print(type(bnf))
# bnf

In [43]:
# bnf['datetime'] = pd.to_datetime(bnf['datetime'])
# bnf = bnf.drop(columns=['Unnamed: 0'])
# bnf.set_index(bnf['datetime'], inplace=True)
# bnf
# bnf = bnf.with_columns(pl.col('datetime').str.to_datetime(format='%Y-%m-%dT%H:%M:%S.%f'))
# print(bnf)
# bnf = bnf.with_columns(pl.col('datetime').cast(pl.DateTime))

# Set 'datetime' column as index
bnf = bnf.with_columns([pl.col("datetime").alias("index")]).drop("datetime")

# Now 'datetime' is set as the index
# bnf

In [44]:
bnf = bnf.with_columns(pl.col("index").alias("datetime"))
# bnf

In [45]:
bnf.tail()

open,high,low,close,volume,index,datetime
f64,f64,f64,f64,i64,datetime[ns],datetime[ns]
49800.45,49812.75,49792.7,49806.2,0,2024-06-07 15:25:00,2024-06-07 15:25:00
49807.25,49811.7,49798.75,49807.5,0,2024-06-07 15:26:00,2024-06-07 15:26:00
49810.85,49823.8,49797.6,49817.3,0,2024-06-07 15:27:00,2024-06-07 15:27:00
49816.25,49827.6,49805.8,49816.0,0,2024-06-07 15:28:00,2024-06-07 15:28:00
49820.3,49825.0,49805.25,49806.6,0,2024-06-07 15:29:00,2024-06-07 15:29:00


In [80]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("o").first().alias("o"),
                pl.col("h").max().alias("h"),
                pl.col("l").min().alias("l"),
                pl.col("c").last().alias("c"),
                # pl.col("volume").sum().alias("volume"),
            ]
        )
    )

# ohlc_resampled = resample(bnf, '60m', pd.Timedelta(minutes=15))

# bnf_df = bnf.to_pandas()
# bnf_df.set_index(bnf_df['datetime'], inplace=True)
# ohlc_15min = bnf_df.resample('60min').agg({
#     'open': 'first',
#     'high': 'max',
#     'low': 'min',
#     'close': 'last',
# })

# ohlc_15min.dropna(inplace=True)

# ohlc_15min.reset_index(inplace=True)

# bnf_1hr = ohlc_15min
# bnf_1hr = ohlc_resampled
# bnf_final = bnf_1hr.to_pandas()
# bnf_final['datetime'] = pd.to_datetime(bnf_final['datetime'])
# bnf_final
# bnf_1hr

In [47]:
# bnf_final = bnf
# bnf_final

In [48]:
# # bnf_final = bnf_1hr
# bnf_final['datetime'] = pd.to_datetime(bnf_final['datetime'])
# bnf_final

In [156]:
import pandas as pd
import numpy as np

def linear_regression_vectorized(df, period, trailing):
    
    if 'c' not in df.columns:
        print(df.columns)
        raise ValueError("The DataFrame must contain a 'close' column.")

    df['c'] = df['c'].astype(float)
    x = np.arange(period)
    x_sum = np.sum(x)
    x_squared_sum = np.sum(x**2)
    
    denominator = period * x_squared_sum - x_sum**2
    
    y_rolling = df['c'].rolling(window=period)
    
    y_sum = y_rolling.sum()
    xy_sum = y_rolling.apply(lambda y: np.dot(x, y), raw=True)
    
    slope = (period * xy_sum - x_sum * y_sum) / denominator
    intercept = (y_sum - slope * x_sum) / period
    
    predicted_values = intercept + slope * (period - 1)
    df['LR'] = predicted_values

    x = np.arange(trailing)
    x_sum = np.sum(x)
    x_squared_sum = np.sum(x**2)
    
    denominator = trailing * x_squared_sum - x_sum**2
    
    y_rolling = df['c'].rolling(window=trailing)
    
    y_sum = y_rolling.sum()
    xy_sum = y_rolling.apply(lambda y: np.dot(x, y), raw=True)
    
    slope = (trailing * xy_sum - x_sum * y_sum) / denominator
    intercept = (y_sum - slope * x_sum) / trailing
    
    predicted_values = intercept + slope * (trailing - 1)
    df['LR2'] = predicted_values
    
    return df

# data = linear_regression_vectorized(bnf.to_pandas(), 20)

In [53]:
# Positional

def execute(df, n):

    trade_book = []
    in_trade = False
    signal_entry_price = 100000
    signal_initial_sl = 0
    already_signal_exists = False
    is_trailing_active = False
    remark = ""
    portfolio_value = 1_00_00_000

    for i in range(1, len(df)):
        points = 0
        current_candle_open = df.iloc[i]["o"]
        current_candle_high = df.iloc[i]["h"]
        current_candle_low = df.iloc[i]["l"]
        current_candle_close = df.iloc[i]["c"]
        current_linear_regression = df.iloc[i]['LR2']

        if not in_trade:
            if df.iloc[i]["Buy_Signal"] == 1:
                if not already_signal_exists:
                    # Fresh Buy Signal
                    # print(df.iloc[i])
                    # print('Fresh Buy Signal')
                    signal_entry_price = current_candle_high
                    # signal_initial_sl = current_candle_low
                    signal_initial_sl = df.iloc[i-n+1:i+1]['l'].min()
                    signal_creation_time = df.iloc[i]["datetime"]
                    already_signal_exists = True

                else:
                    if current_candle_open > signal_entry_price:
                        # Gap Up Open, SKIP trade
                        # print(df.iloc[i])
                        # print('Gap Up Open, Skip Trade')
                        already_signal_exists = False
                        signal_entry_price = 100000
                        signal_initial_sl = 0
                    elif current_candle_high < signal_entry_price:
                        # Better Candle
                        # print(df.iloc[i])
                        # print('Better Candle')
                        signal_entry_price = current_candle_high
                        # signal_initial_sl = current_candle_low
                        signal_initial_sl = df.iloc[i-n+1:i+1]['l'].min()
                        signal_creation_time = df.iloc[i]["datetime"]

                        # if (signal_entry_price - signal_initial_sl > 400):
                        #     # Skip Signal Candle Due To Big Size
                        #     already_signal_exists = False
                        #     signal_entry_price = 100000
                        #     signal_initial_sl = 0

                    elif current_candle_high > signal_entry_price:
                        # Entry Triggered
                        # print(df.iloc[i])
                        # print('Entry Triggered')
                        in_trade = True
                        entry_time = df.iloc[i]["datetime"]
                        entry_price = signal_entry_price
                        points = 0
            else:
                if df.iloc[i - 1]["Buy_Signal"] == 1:
                    # Considering the forward bias condition as well now
                    signal_entry_price = df.iloc[i - 1]["h"]
                    # signal_initial_sl = df.iloc[i - 1]["l"]
                    signal_initial_sl = df.iloc[i-n:i]['l'].min()
                    signal_creation_time = df.iloc[i - 1]["datetime"]
                    already_signal_exists = True

                    if current_candle_high > signal_entry_price:
                        in_trade = True
                        entry_time = df.iloc[i]["datetime"]
                        entry_price = signal_entry_price
                        points = 0
                    else:
                        # Discard Existing Signal
                        already_signal_exists = False
                        signal_entry_price = 100000
                        signal_initial_sl = 0

        if in_trade:
            trade_entry_price = signal_entry_price
            trade_initial_sl = signal_initial_sl
            trade_final_sl = signal_initial_sl

            if (
                not is_trailing_active
                and current_candle_low > current_linear_regression
            ):
                is_trailing_active = True

            if not is_trailing_active:
                if current_candle_open < trade_initial_sl:
                    # if df.iloc[i-1]['l'] < trade_initial_sl:
                    #     #Previous Candle Already Hit the SL Hence don't check gap down
                    #     print(df.iloc[i])
                    #     print('Initial SL Hit Before Gap Down')
                    #     in_trade = False
                    #     points = trade_initial_sl - trade_entry_price
                    #     exit_price = trade_initial_sl
                    #     exit_time = df.iloc[i-1]['datetime']
                    #     remark = 'Initial SL Hit Before Gap Down'
                    # else:
                    if (
                        df.iloc[i]["datetime"].date() == entry_time.date()
                        and df.iloc[i]["datetime"].time() == entry_time.time()
                    ):
                        if current_candle_close <= trade_initial_sl:
                            in_trade = False
                            points = trade_initial_sl - trade_entry_price
                            exit_price = trade_initial_sl
                            exit_time = df.iloc[i]["datetime"]
                            remark = "Initial SL hit"

                    else:
                        # Gap Open Outside ISL
                        # print(df.iloc[i])
                        # print('Gap Open Outside ISL')
                        in_trade = False
                        points = current_candle_open - trade_entry_price
                        exit_price = current_candle_open
                        exit_time = df.iloc[i]["datetime"]
                        remark = "Gap Open Outside ISL"

                elif current_candle_low <= trade_initial_sl:
                    # Initial SL Hit
                    # print(df.iloc[i])
                    # print('Initial SL Hit')
                    in_trade = False
                    points = trade_initial_sl - trade_entry_price
                    exit_price = trade_initial_sl
                    exit_time = df.iloc[i]["datetime"]
                    remark = "Initial SL Hit"

            else:
                trade_final_sl = max(trade_initial_sl, current_linear_regression)

                if current_candle_open <= trade_initial_sl:
                    # if df.iloc[i-1]['l'] < trade_initial_sl:
                    #     #Previous Candle Already Hit the SL Hence don't check gap down
                    #     print(df.iloc[i])
                    #     print('Initial SL Hit Before Gap Down')
                    #     in_trade = False
                    #     points = trade_initial_sl - trade_entry_price
                    #     exit_price = trade_initial_sl
                    #     exit_time = df.iloc[i-1]['datetime']
                    #     remark = 'Initial SL Hit'
                    # else:
                    if (
                        df.iloc[i]["datetime"].date() == entry_time.date()
                        and df.iloc[i]["datetime"].time() == entry_time.time()
                    ):
                        if current_candle_close <= trade_initial_sl:
                            in_trade = False
                            points = trade_initial_sl - trade_entry_price
                            exit_price = trade_initial_sl
                            exit_time = df.iloc[i]["datetime"]
                            remark = "Initial SL hit"

                    else:
                        # Gap Open Outside ISL
                        # print(df.iloc[i])
                        # print('Gap Open Outside ISL')
                        in_trade = False
                        points = current_candle_open - trade_entry_price
                        exit_price = current_candle_open
                        exit_time = df.iloc[i]["datetime"]
                        remark = "Gap Open Outside ISL"

                elif current_candle_low <= trade_initial_sl:
                    # if trade_initial_sl >= current_linear_regression:
                    # Despite Trailing, Initial SL hit
                    # print(df.iloc[i])
                    # print('Initial SL Hit')
                    in_trade = False
                    points = trade_initial_sl - trade_entry_price
                    exit_price = trade_initial_sl
                    exit_time = df.iloc[i]["datetime"]
                    remark = "Initial SL hit"

                elif current_candle_close <= trade_final_sl:
                    # Price Closed below TSL i.e. MA10 , TSL Hit
                    # print(df.iloc[i])
                    # print('Initial SL Hit')
                    in_trade = False
                    points = current_candle_close - trade_entry_price
                    exit_price = current_candle_close
                    exit_time = df.iloc[i]["datetime"]
                    is_trailing_active = False
                    remark = "TSL Hit"

            if points:
                index_lev = 6
                qty = int(round(portfolio_value * LEVERAGE_ / entry_price / LOT_SIZE_)) * LOT_SIZE_
                slippage = SLIPPAGE_ * (entry_price + exit_price)
                # slippage = 10
                final_points = points - slippage
                # final_points = points
                trade = {
                    "Signal Generated At": signal_creation_time,
                    "Trade Type": "LONG",
                    "Entry Time": entry_time,
                    "Entry Price": entry_price,
                    "Initial SL": trade_initial_sl,
                    "Final SL": trade_final_sl,
                    "Exit Time": exit_time,
                    "Exit Price": exit_price,
                    "Points Captured": points,
                    "After Costs": final_points,
                    "PnL": final_points * qty,
                    "Remarks": remark,
                    "Qty": qty,
                    "Leverage": index_lev,
                    "ROI%": (final_points * qty / portfolio_value) * 100,
                    "Trade Year": entry_time.year,
                    "Trade Month": entry_time.month,
                }
                trade_book.append(trade)
                points = 0
                in_trade = False
                already_signal_exists = False
                remark = ""
                is_trailing_active = False

    trade_book_df = pd.DataFrame(trade_book)
    return trade_book_df

In [54]:
# Positional Short

def execute_short(df, n):

    trade_book = []
    in_trade = False
    signal_entry_price = 100000
    signal_initial_sl = 0
    already_signal_exists = False
    is_trailing_active = False
    remark = ""
    portfolio_value = 1_00_00_000

    for i in range(1, len(df)):
        points = 0
        current_candle_open = df.iloc[i]["o"]
        current_candle_high = df.iloc[i]["h"]
        current_candle_low = df.iloc[i]["l"]
        current_candle_close = df.iloc[i]["c"]
        current_linear_regression = df.iloc[i]['LR2']

        if not in_trade:
            if df.iloc[i]["Sell_Signal"] == 1:
                if not already_signal_exists:
                    # Fresh Buy Signal
                    # print(df.iloc[i])
                    # print('Fresh Buy Signal')
                    signal_entry_price = current_candle_low
                    # signal_initial_sl = current_candle_low
                    signal_initial_sl = df.iloc[i-n+1:i+1]['h'].max()
                    signal_creation_time = df.iloc[i]["datetime"]
                    already_signal_exists = True

                else:
                    if current_candle_open < signal_entry_price:
                        # Gap Down Open, SKIP trade
                        # print(df.iloc[i])
                        # print('Gap Up Open, Skip Trade')
                        already_signal_exists = False
                        signal_entry_price = 100000
                        signal_initial_sl = 0
                    elif current_candle_low > signal_entry_price:
                        # Better Candle
                        # print(df.iloc[i])
                        # print('Better Candle')
                        signal_entry_price = current_candle_low
                        # signal_initial_sl = current_candle_low
                        signal_initial_sl = df.iloc[i-n+1:i+1]['h'].max()
                        signal_creation_time = df.iloc[i]["datetime"]

                        # if (signal_entry_price - signal_initial_sl > 400):
                        #     # Skip Signal Candle Due To Big Size
                        #     already_signal_exists = False
                        #     signal_entry_price = 100000
                        #     signal_initial_sl = 0

                    elif current_candle_low <= signal_entry_price:
                        # Entry Triggered
                        # print(df.iloc[i])
                        # print('Entry Triggered')
                        in_trade = True
                        entry_time = df.iloc[i]["datetime"]
                        entry_price = signal_entry_price
                        points = 0
            else:
                if df.iloc[i - 1]["Sell_Signal"] == 1:
                    # Considering the forward bias condition as well now
                    signal_entry_price = df.iloc[i - 1]["l"]
                    # signal_initial_sl = df.iloc[i - 1]["l"]
                    signal_initial_sl = df.iloc[i-n:i]['h'].max()
                    signal_creation_time = df.iloc[i - 1]["datetime"]
                    already_signal_exists = True

                    if current_candle_low <= signal_entry_price:
                        in_trade = True
                        entry_time = df.iloc[i]["datetime"]
                        entry_price = signal_entry_price
                        points = 0
                    else:
                        # Discard Existing Signal
                        already_signal_exists = False
                        signal_entry_price = 100000
                        signal_initial_sl = 0

        if in_trade:
            trade_entry_price = signal_entry_price
            trade_initial_sl = signal_initial_sl
            trade_final_sl = signal_initial_sl

            if (
                not is_trailing_active
                and current_candle_high < current_linear_regression
            ):
                is_trailing_active = True

            if not is_trailing_active:
                if current_candle_open >= trade_initial_sl:
                    # if df.iloc[i-1]['l'] < trade_initial_sl:
                    #     #Previous Candle Already Hit the SL Hence don't check gap down
                    #     print(df.iloc[i])
                    #     print('Initial SL Hit Before Gap Down')
                    #     in_trade = False
                    #     points = trade_initial_sl - trade_entry_price
                    #     exit_price = trade_initial_sl
                    #     exit_time = df.iloc[i-1]['datetime']
                    #     remark = 'Initial SL Hit Before Gap Down'
                    # else:
                    if (
                        df.iloc[i]["datetime"].date() == entry_time.date()
                        and df.iloc[i]["datetime"].time() == entry_time.time()
                    ):
                        in_trade = False
                        points = trade_entry_price - trade_initial_sl
                        exit_price = trade_initial_sl
                        exit_time = df.iloc[i]["datetime"]
                        remark = "Initial SL hit"

                    else:
                        # Gap Open Outside ISL
                        # print(df.iloc[i])
                        # print('Gap Open Outside ISL')
                        in_trade = False
                        points = trade_entry_price - current_candle_open
                        exit_price = current_candle_open
                        exit_time = df.iloc[i]["datetime"]
                        remark = "Gap Open Outside ISL"

                elif current_candle_high >= trade_initial_sl:
                    # Initial SL Hit
                    # print(df.iloc[i])
                    # print('Initial SL Hit')
                    in_trade = False
                    points = trade_entry_price - trade_initial_sl
                    exit_price = trade_initial_sl
                    exit_time = df.iloc[i]["datetime"]
                    remark = "Initial SL Hit"

            else:
                trade_final_sl = min(trade_initial_sl, current_linear_regression)

                if current_candle_open >= trade_initial_sl:
                    # if df.iloc[i-1]['l'] < trade_initial_sl:
                    #     #Previous Candle Already Hit the SL Hence don't check gap down
                    #     print(df.iloc[i])
                    #     print('Initial SL Hit Before Gap Down')
                    #     in_trade = False
                    #     points = trade_initial_sl - trade_entry_price
                    #     exit_price = trade_initial_sl
                    #     exit_time = df.iloc[i-1]['datetime']
                    #     remark = 'Initial SL Hit'
                    # else:
                    if (
                        df.iloc[i]["datetime"].date() == entry_time.date()
                        and df.iloc[i]["datetime"].time() == entry_time.time()
                    ):
                        in_trade = False
                        points = trade_entry_price - trade_initial_sl
                        exit_price = trade_initial_sl
                        exit_time = df.iloc[i]["datetime"]
                        remark = "Initial SL hit"

                    else:
                        # Gap Open Outside ISL
                        # print(df.iloc[i])
                        # print('Gap Open Outside ISL')
                        in_trade = False
                        points = trade_entry_price - current_candle_open
                        exit_price = current_candle_open
                        exit_time = df.iloc[i]["datetime"]
                        remark = "Gap Open Outside ISL"

                elif current_candle_high >= trade_initial_sl:
                    # if trade_initial_sl >= current_linear_regression:
                    # Despite Trailing, Initial SL hit
                    # print(df.iloc[i])
                    # print('Initial SL Hit')
                    in_trade = False
                    points = trade_entry_price - trade_initial_sl
                    exit_price = trade_initial_sl
                    exit_time = df.iloc[i]["datetime"]
                    remark = "Initial SL hit"

                elif current_candle_close >= trade_final_sl:
                    # Price Closed below TSL i.e. MA10 , TSL Hit
                    # print(df.iloc[i])
                    # print('Initial SL Hit')
                    in_trade = False
                    points = trade_entry_price - current_candle_close
                    exit_price = current_candle_close
                    exit_time = df.iloc[i]["datetime"]
                    is_trailing_active = False
                    remark = "TSL Hit"

            if points:
                index_lev = 6
                qty = int(round(portfolio_value * index_lev / entry_price / 25)) * 25
                slippage = 0.0001 * (entry_price + exit_price)
                # slippage = 10
                final_points = points - slippage
                # final_points = points
                trade = {
                    "Signal Generated At": signal_creation_time,
                    "Trade Type": "SHORT",
                    "Entry Time": entry_time,
                    "Entry Price": entry_price,
                    "Initial SL": trade_initial_sl,
                    "Final SL": trade_final_sl,
                    "Exit Time": exit_time,
                    "Exit Price": exit_price,
                    "Points Captured": points,
                    "After Costs": final_points,
                    "PnL": final_points * qty,
                    "Remarks": remark,
                    "Qty": qty,
                    "Leverage": index_lev,
                    "ROI%": (final_points * qty / portfolio_value) * 100,
                    "Trade Year": entry_time.year,
                    "Trade Month": entry_time.month,
                }
                trade_book.append(trade)
                points = 0
                in_trade = False
                already_signal_exists = False
                remark = ""
                is_trailing_active = False

    trade_book_df = pd.DataFrame(trade_book)
    return trade_book_df

In [327]:
# Option Selling

async def option_selling(df, index, strike, asset_class, expiry, trade_date, sl_pct):

    trade_book = []
    in_trade = False
    signal_entry_price = 100000
    signal_initial_sl = 0
    already_signal_exists = False
    is_trailing_active = False
    remark = ""
    portfolio_value = 1_00_00_000

    df['datetime'] = pd.to_datetime(df['datetime'])

    df = df[df['datetime'].dt.date >= trade_date]

    # print(df.head(25).to_string())

    for i in range(2, len(df)):
        points = 0
        current_candle_open = df.iloc[i]["o"]
        current_candle_high = df.iloc[i]["h"]
        current_candle_low = df.iloc[i]["l"]
        current_candle_close = df.iloc[i]["c"]
        current_linear_regression = df.iloc[i]['LR2']
        # print(df.iloc[i])

        if not in_trade:
            if df.iloc[i-1]['LR'] < df.iloc[i-2]['LR']:
                # Previous Candle has a signal
                signal_entry_price = df.iloc[i-1]['l']
                signal_initial_sl = signal_entry_price * (1 + sl_pct)
                signal_generation_time = df.iloc[i-1]['datetime']

                if current_candle_low <= signal_entry_price:
                    if current_candle_open <= signal_entry_price:
                        if current_candle_high >= signal_entry_price:
                            # Entry Triggered
                            in_trade = True
                            entry_time = df.iloc[i]['datetime']
                            entry_price = signal_entry_price
                            points = 0
                        else:
                            # Trade Skipped, Gap Open Outside Entry
                            continue
                    else:
                        # Entry Triggered
                        in_trade = True
                        entry_time = df.iloc[i]['datetime']
                        entry_price = signal_entry_price
                        points = 0
                else:
                    # Check Next Iteration for Better Candle Scenario
                    continue

        if in_trade:
            trade_entry_price = signal_entry_price
            trade_initial_sl = signal_initial_sl
            trade_final_sl = signal_initial_sl
            
            if (
                not is_trailing_active
                and current_candle_high < current_linear_regression
            ):
                is_trailing_active = True

            if not is_trailing_active:
                if current_candle_open >= signal_initial_sl:
                    if (
                        df.iloc[i]["datetime"].date() == entry_time.date()
                        and df.iloc[i]["datetime"].time() == entry_time.time()
                    ):
                        # Check if this Gap Open below SL candle is the Entry Candle
                        if current_candle_close >= trade_initial_sl:
                            in_trade = False
                            points = trade_entry_price - trade_initial_sl
                            exit_price = trade_initial_sl
                            exit_time = df.iloc[i]["datetime"]
                            remark = "Initial SL Hit"

                    # else:
                    #     # Gap Open Outside ISL
                    #     in_trade = False
                    #     points = trade_entry_price - current_candle_open
                    #     exit_price = current_candle_open
                    #     exit_time = df.iloc[i]["datetime"]
                    #     remark = "Gap Open Outside ISL"

                elif current_candle_high >= trade_initial_sl:
                    # Initial SL Hit
                    in_trade = False
                    points = trade_entry_price - trade_initial_sl
                    exit_price = trade_initial_sl
                    exit_time = df.iloc[i]["datetime"]
                    remark = "Initial SL Hit"

                elif df.iloc[i]['datetime'] >= dt.datetime.combine(trade_date, dt.time(15, 20)):
                    in_trade = False
                    points = trade_entry_price - current_candle_close
                    exit_price = current_candle_close
                    exit_time = df.iloc[i]["datetime"]
                    remark = "EOD Exit"

            else:
                trade_final_sl = min(trade_initial_sl, current_linear_regression)

                if current_candle_open >= trade_initial_sl:
                    if (
                        df.iloc[i]["datetime"].date() == entry_time.date()
                        and df.iloc[i]["datetime"].time() == entry_time.time()
                    ):
                        if current_candle_close >= trade_initial_sl:
                            in_trade = False
                            points = trade_entry_price - trade_initial_sl
                            exit_price = trade_initial_sl
                            exit_time = df.iloc[i]["datetime"]
                            remark = "Initial SL Hit"

                    # else:
                    #     # Gap Open Outside ISL
                    #     in_trade = False
                    #     points = trade_entry_price - current_candle_open
                    #     exit_price = current_candle_open
                    #     exit_time = df.iloc[i]["datetime"]
                    #     remark = "Gap Open Outside ISL"

                elif current_candle_high >= trade_initial_sl:
                    # Despite Trailing, Initial SL hit
                    in_trade = False
                    points = trade_entry_price - trade_initial_sl
                    exit_price = trade_initial_sl
                    exit_time = df.iloc[i]["datetime"]
                    remark = "Initial SL Hit"

                elif current_candle_close >= trade_final_sl:
                    # Price Closed below TSL
                    in_trade = False
                    points = trade_entry_price - current_candle_close
                    exit_price = current_candle_close
                    exit_time = df.iloc[i]["datetime"]
                    is_trailing_active = False
                    remark = "TSL Hit"

                elif df.iloc[i]['datetime'] >= dt.datetime.combine(trade_date, dt.time(15, 20)):
                    in_trade = False
                    points = trade_entry_price - current_candle_close
                    exit_price = current_candle_close
                    exit_time = df.iloc[i]["datetime"]
                    remark = "EOD Exit"

            if points:
                qty = int(round((PORTFOLIO * LEVERAGE_ / strike) / LOT_SIZE_)) * LOT_SIZE_
                slippage = SLIPPAGE_ * (entry_price + exit_price)
                # slippage = 10
                final_points = points - slippage
                # final_points = points
                trade = {
                    "Trade Type": "SHORT",
                    "Index": index,
                    "Strike": strike,
                    "Asset Class": asset_class,
                    "Expiry": expiry,
                    "Signal Generated At": signal_generation_time,
                    "Entry Time": entry_time,
                    "Entry Price": entry_price,
                    "Initial SL": trade_initial_sl,
                    "Final SL": trade_final_sl,
                    "Exit Time": exit_time,
                    "Exit Price": exit_price,
                    "Points Captured": points,
                    "After Costs": final_points,
                    "PnL": final_points * qty,
                    "Remarks": remark,
                    "Qty": qty,
                    "Leverage": LEVERAGE_,
                    "ROI%": (final_points * qty / PORTFOLIO) * 100,
                    "Trade Year": entry_time.year,
                    "Trade Month": entry_time.month,
                }
                # print(trade)
                trade_book.append(trade)
                points = 0
                in_trade = False
                already_signal_exists = False
                remark = ""
                is_trailing_active = False

    trade_book_df = pd.DataFrame(trade_book)
    return trade_book_df

In [256]:
bnf1 = bnf
bnf2 = bnf

In [267]:
def generate_stats(tb_expiry, variation):
    stats_df8 = pd.DataFrame(
        index=range(2017, 2025),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2017, 2025):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = variation
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0]["ROI%"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum() - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation
    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    # print(stats_df8)
    return {overall_roi_dd_ratio : stats_df8}

In [324]:
def generate_stats_by_dte(tb_expiry, variation):
    # Create a DataFrame to store the statistics
    stats_df8 = pd.DataFrame(
        columns=[
            "DTE",
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ]
    )

    # Group the data by 'DTE'
    grouped_by_dte = tb_expiry.groupby("DTE")

    # Iterate over each DTE group
    for dte_value, dte_trades in grouped_by_dte:
        # Calculate total ROI for the current DTE group
        total_roi = dte_trades["ROI%"].sum()

        # Calculate total number of trades for the current DTE group
        total_trades = len(dte_trades)

        # Calculate win rate for the current DTE group
        win_rate = (dte_trades["ROI%"] > 0).mean() * 100

        # Calculate average profit per trade
        avg_profit = dte_trades[dte_trades["ROI%"] > 0]["ROI%"].mean()

        # Calculate average loss per trade
        avg_loss = dte_trades[dte_trades["ROI%"] < 0]["ROI%"].mean()

        # Calculate maximum drawdown for the current DTE group
        max_drawdown = (
            dte_trades["ROI%"].cumsum() - dte_trades["ROI%"].cumsum().cummax()
        ).min()

        # Calculate ROI/DD ratio for the current DTE group
        roi_dd_ratio = total_roi / abs(max_drawdown) if max_drawdown != 0 else 0

        # Store the statistics in the DataFrame for the current DTE
        stats_df8 = pd.concat([stats_df8, pd.DataFrame({
            "DTE": [dte_value],
            "Total ROI": [total_roi],
            "Total Trades": [total_trades],
            "Win Rate": [win_rate],
            "Avg Profit% per Trade": [avg_profit],
            "Avg Loss% per Trade": [avg_loss],
            "Max Drawdown": [max_drawdown],
            "ROI/DD Ratio": [roi_dd_ratio],
            "Variation": [variation],
        })], ignore_index=True)

    # Overall statistics across all DTE groups
    overall_total_roi = tb_expiry["ROI%"].sum()
    overall_total_trades = len(tb_expiry)
    overall_win_rate = (tb_expiry["ROI%"] > 0).mean() * 100
    overall_avg_profit = tb_expiry[tb_expiry["ROI%"] > 0]["ROI%"].mean()
    overall_avg_loss = tb_expiry[tb_expiry["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        tb_expiry["ROI%"].cumsum() - tb_expiry["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown) if overall_max_drawdown != 0 else 0
    overall_variation = variation

    # Store the overall statistics in the DataFrame
    stats_df8 = pd.concat([stats_df8, pd.DataFrame({
        "DTE": ["Overall"],
        "Total ROI": [overall_total_roi],
        "Total Trades": [overall_total_trades],
        "Win Rate": [overall_win_rate],
        "Avg Profit% per Trade": [overall_avg_profit],
        "Avg Loss% per Trade": [overall_avg_loss],
        "Max Drawdown": [overall_max_drawdown],
        "ROI/DD Ratio": [overall_roi_dd_ratio],
        "Variation": [overall_variation],
    })], ignore_index=True)

    return stats_df8

def generate_stats_by_remarks(tb_expiry, variation):
    # Create a DataFrame to store the statistics
    stats_df8 = pd.DataFrame(
        columns=[
            "Remarks",
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ]
    )

    # Group the data by 'Remarks'
    grouped_by_remarks = tb_expiry.groupby("Remarks")

    # Iterate over each group
    for remarks_value, remarks_trades in grouped_by_remarks:
        # Calculate total ROI for the current group
        total_roi = remarks_trades["ROI%"].sum()

        # Calculate total number of trades for the current group
        total_trades = len(remarks_trades)

        # Calculate win rate for the current group
        win_rate = (remarks_trades["ROI%"] > 0).mean() * 100

        # Calculate average profit per trade
        avg_profit = remarks_trades[remarks_trades["ROI%"] > 0]["ROI%"].mean()

        # Calculate average loss per trade
        avg_loss = remarks_trades[remarks_trades["ROI%"] < 0]["ROI%"].mean()

        # Calculate maximum drawdown for the current group
        max_drawdown = (
            remarks_trades["ROI%"].cumsum() - remarks_trades["ROI%"].cumsum().cummax()
        ).min()

        # Calculate ROI/DD ratio for the current group
        roi_dd_ratio = total_roi / abs(max_drawdown) if max_drawdown != 0 else 0

        # Store the statistics in the DataFrame for the current group
        stats_df8 = pd.concat([stats_df8, pd.DataFrame({
            "Remarks": [remarks_value],
            "Total ROI": [total_roi],
            "Total Trades": [total_trades],
            "Win Rate": [win_rate],
            "Avg Profit% per Trade": [avg_profit],
            "Avg Loss% per Trade": [avg_loss],
            "Max Drawdown": [max_drawdown],
            "ROI/DD Ratio": [roi_dd_ratio],
            "Variation": [variation],
        })], ignore_index=True)

    # Overall statistics across all groups
    overall_total_roi = tb_expiry["ROI%"].sum()
    overall_total_trades = len(tb_expiry)
    overall_win_rate = (tb_expiry["ROI%"] > 0).mean() * 100
    overall_avg_profit = tb_expiry[tb_expiry["ROI%"] > 0]["ROI%"].mean()
    overall_avg_loss = tb_expiry[tb_expiry["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        tb_expiry["ROI%"].cumsum() - tb_expiry["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown) if overall_max_drawdown != 0 else 0
    overall_variation = variation

    # Store the overall statistics in the DataFrame
    stats_df8 = pd.concat([stats_df8, pd.DataFrame({
        "Remarks": ["Overall"],
        "Total ROI": [overall_total_roi],
        "Total Trades": [overall_total_trades],
        "Win Rate": [overall_win_rate],
        "Avg Profit% per Trade": [overall_avg_profit],
        "Avg Loss% per Trade": [overall_avg_loss],
        "Max Drawdown": [overall_max_drawdown],
        "ROI/DD Ratio": [overall_roi_dd_ratio],
        "Variation": [overall_variation],
    })], ignore_index=True)

    return stats_df8


In [258]:
bnf1 = bnf1.to_pandas()

In [259]:
# # bnf2 = bnf2.to_pandas()
# data = linear_regression_vectorized(bnf1, 80)
# signals_df1 = generate_signals1(data, 1)
# # signals_df2 = generate_signals2(bnf2)

In [260]:
# data[data['Buy_Signal'] > 0].tail()

In [261]:
bnf1.tail()

,open,high,low,close,volume,index,datetime
687621,49800.4500,49812.7500,49792.7000,49806.2000,0,2024-06-07 15:25:00,2024-06-07 15:25:00
687622,49807.2500,49811.7000,49798.7500,49807.5000,0,2024-06-07 15:26:00,2024-06-07 15:26:00
687623,49810.8500,49823.8000,49797.6000,49817.3000,0,2024-06-07 15:27:00,2024-06-07 15:27:00
687624,49816.2500,49827.6000,49805.8000,49816.0000,0,2024-06-07 15:28:00,2024-06-07 15:28:00
687625,49820.3000,49825.0000,49805.2500,49806.6000,0,2024-06-07 15:29:00,2024-06-07 15:29:00


In [275]:
async def trade(df, lr_1, lr_2, sl_pct, tf, ofs):

    total_trades = pd.DataFrame()
    
    start_dt = dt.date(2017, 1, 1)
    end_dt = dt.date(2024, 9, 30)
    current_dt = start_dt

    while current_dt <= end_dt:
        
        combined_trades_ce = pd.DataFrame()
        combined_trades_pe = pd.DataFrame()
        
        # print(current_dt)

        spot = df.loc[df['datetime'].dt.date == current_dt, 'open']
        if len(spot) == 0:                                                                # This will give us 9:15 open
            current_dt += dt.timedelta(days=1)
            continue
        else:
            spot_open = spot.iloc[0]
            # print(spot_open)
            # break

            atm_strike = int(round(spot_open / SPREAD_) * SPREAD_)
            # print(atm_strike)

            current_expiry = await get_expiry(current_dt, symbol)

            ce_df = await fetch_data(
                index=symbol,
                start_date=current_dt - dt.timedelta(days=3),
                end_date=current_dt,
                start_time=dt.time(9, 15),
                end_time=dt.time(15, 30),
                expiry=current_expiry,
                strike=atm_strike,
                asset_class="C",
            )

            if not isinstance(ce_df, str) and ce_df is not None:
                ce_df = ce_df.select(["datetime", "o", "h", "l", "c", "v"])
                ce_df = resample(ce_df, tf, offset=ofs)
                ce_df_pandas = ce_df.to_pandas()
                ce_df_pandas = ce_df_pandas[~(ce_df_pandas['datetime'].dt.time == pd.to_datetime('15:30').time())]
                ce_df_pandas = linear_regression_vectorized(ce_df_pandas, lr_1, lr_2)

            else:
                ce_df_pandas = pd.DataFrame()

            pe_df = await fetch_data(
                index=symbol,
                start_date=current_dt - dt.timedelta(days=3),
                end_date=current_dt,
                start_time=dt.time(9, 15),
                end_time=dt.time(15, 30),
                expiry=current_expiry,
                strike=atm_strike,
                asset_class="P",
            )

            if not isinstance(pe_df, str) and pe_df is not None:
                pe_df = pe_df.select(["datetime", "o", "h", "l", "c", "v"])
                pe_df = resample(pe_df, tf, offset=ofs)
                pe_df_pandas = pe_df.to_pandas()
                pe_df_pandas = pe_df_pandas[~(pe_df_pandas['datetime'].dt.time == pd.to_datetime('15:30').time())]
                pe_df_pandas = linear_regression_vectorized(pe_df_pandas, lr_1, lr_2)

            else:
                pe_df_pandas = pd.DataFrame()

            if len(ce_df_pandas) > 0:
                ce_trades = await option_selling(
                    df=ce_df_pandas,
                    index=symbol,
                    strike=atm_strike,
                    asset_class="C",
                    expiry=current_expiry,
                    trade_date=current_dt,
                    sl_pct=sl_pct,
                )
                combined_trades_ce = pd.concat([combined_trades_ce, ce_trades], ignore_index=True)

            if len(pe_df_pandas) > 0:
                pe_trades = await option_selling(
                    df=pe_df_pandas,
                    index=symbol,
                    strike=atm_strike,
                    asset_class="P",
                    expiry=current_expiry,
                    trade_date=current_dt,
                    sl_pct=sl_pct,
                )
                combined_trades_pe = pd.concat([combined_trades_pe, pe_trades], ignore_index=True)

        total_trades = pd.concat([total_trades, combined_trades_ce, combined_trades_pe], ignore_index=True)
        if len(total_trades) > 0:
            total_trades = total_trades.sort_values(by="Signal Generated At", ignore_index=True)
            
        current_dt += dt.timedelta(days=1)
    return total_trades
        

In [338]:
tb = await trade(bnf1, 11, 21, 0.75, '10m', '5m')

In [339]:
tb['Entry Time'] = pd.to_datetime(tb['Entry Time'])
tb['Expiry'] = pd.to_datetime(tb['Expiry'])
tb['DTE'] = (tb['Expiry'] - tb['Entry Time']).dt.days + 1

In [340]:
stats = generate_stats_by_dte(tb , '...')
print(stats.to_string())

       DTE  Total ROI Total Trades  Win Rate  Avg Profit% per Trade  Avg Loss% per Trade  Max Drawdown  ROI/DD Ratio Variation
0        0   157.5834         2103   53.7803                 0.7200              -0.6757      -31.4024        5.0182       ...
1        1    59.4692         1721   52.7600                 0.8333              -0.8575      -29.2381        2.0340       ...
2        2    68.1663         1602   50.3121                 0.8404              -0.7654      -29.6616        2.2981       ...
3        3   -70.8882         1403   47.1133                 0.7933              -0.8022      -78.0196       -0.9086       ...
4        4     0.2202            2   50.0000                 0.2609              -0.0407       -0.0407        5.4065       ...
5        5   -36.6089          176   51.1364                 0.6816              -1.1390      -39.5651       -0.9253       ...
6        6  -104.5716         1510   47.6821                 0.7859              -0.8487     -110.0559       -0

/tmp/ipykernel_3931/610469842.py:46: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  stats_df8 = pd.concat([stats_df8, pd.DataFrame({


In [341]:
stats = generate_stats_by_remarks(tb , '...')
print(stats.to_string())

          Remarks  Total ROI Total Trades  Win Rate  Avg Profit% per Trade  Avg Loss% per Trade  Max Drawdown  ROI/DD Ratio Variation
0        EOD Exit   751.2001         2209   66.9986                 0.7879              -0.5692       -8.1162       92.5556       ...
1  Initial SL Hit -1137.2733          776    0.0000                    NaN              -1.4656    -1136.3790       -1.0008       ...
2         TSL Hit   460.7684         5536   51.3006                 0.7874              -0.6585      -26.4783       17.4017       ...
3         Overall    74.6951         8521   50.6983                 0.7876              -0.7921     -127.4280        0.5862       ...


/tmp/ipykernel_3931/610469842.py:130: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  stats_df8 = pd.concat([stats_df8, pd.DataFrame({


In [337]:
tb.tail()

,Trade Type,Index,Strike,Asset Class,Expiry,Signal Generated At,Entry Time,Entry Price,Initial SL,Final SL,Exit Time,Exit Price,Points Captured,After Costs,PnL,Remarks,Qty,Leverage,ROI%,Trade Year,Trade Month,DTE
19482,SHORT,bnf,49300,C,2024-06-12,2024-06-07 14:33:00,2024-06-07 14:36:00,783.6000,1057.8600,795.0763,2024-06-07 14:48:00,806.4000,-22.8000,-38.7000,-39474.0000,TSL Hit,1020,5,-0.3947,2024,6,5
19483,SHORT,bnf,49300,P,2024-06-12,2024-06-07 14:45:00,2024-06-07 14:48:00,265.3000,358.1550,266.2050,2024-06-07 14:57:00,271.4500,-6.1500,-11.5175,-11747.8500,TSL Hit,1020,5,-0.1175,2024,6,5
19484,SHORT,bnf,49300,C,2024-06-12,2024-06-07 14:54:00,2024-06-07 14:57:00,806.2500,1088.4375,798.2315,2024-06-07 15:18:00,817.1500,-10.9000,-27.1340,-27676.6800,TSL Hit,1020,5,-0.2768,2024,6,5
19485,SHORT,bnf,49300,P,2024-06-12,2024-06-07 15:15:00,2024-06-07 15:18:00,269.9000,364.3650,364.3650,2024-06-07 15:21:00,263.2500,6.6500,1.3185,1344.8700,EOD Exit,1020,5,0.0134,2024,6,5
19486,SHORT,bnf,49300,P,2024-06-12,2024-06-07 15:24:00,2024-06-07 15:27:00,260.9000,352.2150,352.2150,2024-06-07 15:27:00,260.4000,0.5000,-4.7130,-4807.2600,EOD Exit,1020,5,-0.0481,2024,6,5


In [274]:
stats_dictionary = {}

sls = [0.5, 1.0, 1.5, 2.0]
tfs = ['5m']

for i in range(5, 81, 5):
    for j in range(5, 31, 5):
        for sl in sls:
            for tf in tfs:
                if tf == '10m':
                    ofs = '5m'
                else:
                    ofs = '0m'
                variation = f"LR: {i}, TLR: {j}, SL: {sl*100}%, TF: {tf}"
                print(variation)
                bnf2 = bnf1
                new_tb = await trade(bnf2, i, j, sl, tf, ofs)
                new_tb["DD%"] = new_tb["ROI%"].cumsum() - new_tb["ROI%"].cumsum().cummax()
                tradebook_buy_side = new_tb
                stats = generate_stats(tradebook_buy_side, variation)
                for overall_roi_dd_ratio, stats_df in stats.items():
                    if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > 10:
                        print(stats_df.to_string())
                        stats_dictionary[overall_roi_dd_ratio] = stats_df

LR: 5, TLR: 5, SL: 50.0%, TF: 5m
LR: 5, TLR: 5, SL: 100.0%, TF: 5m
LR: 5, TLR: 5, SL: 150.0%, TF: 5m
LR: 5, TLR: 5, SL: 200.0%, TF: 5m
LR: 5, TLR: 10, SL: 50.0%, TF: 5m
LR: 5, TLR: 10, SL: 100.0%, TF: 5m
LR: 5, TLR: 10, SL: 150.0%, TF: 5m
LR: 5, TLR: 10, SL: 200.0%, TF: 5m
LR: 5, TLR: 15, SL: 50.0%, TF: 5m
LR: 5, TLR: 15, SL: 100.0%, TF: 5m
LR: 5, TLR: 15, SL: 150.0%, TF: 5m
LR: 5, TLR: 15, SL: 200.0%, TF: 5m
LR: 5, TLR: 20, SL: 50.0%, TF: 5m
LR: 5, TLR: 20, SL: 100.0%, TF: 5m
LR: 5, TLR: 20, SL: 150.0%, TF: 5m
LR: 5, TLR: 20, SL: 200.0%, TF: 5m
LR: 5, TLR: 25, SL: 50.0%, TF: 5m
LR: 5, TLR: 25, SL: 100.0%, TF: 5m
LR: 5, TLR: 25, SL: 150.0%, TF: 5m
LR: 5, TLR: 25, SL: 200.0%, TF: 5m
LR: 5, TLR: 30, SL: 50.0%, TF: 5m
LR: 5, TLR: 30, SL: 100.0%, TF: 5m
LR: 5, TLR: 30, SL: 150.0%, TF: 5m
LR: 5, TLR: 30, SL: 200.0%, TF: 5m
LR: 10, TLR: 5, SL: 50.0%, TF: 5m
LR: 10, TLR: 5, SL: 100.0%, TF: 5m
LR: 10, TLR: 5, SL: 150.0%, TF: 5m
LR: 10, TLR: 5, SL: 200.0%, TF: 5m
LR: 10, TLR: 10, SL: 50.0%, TF:

KeyError: 'Signal Generated At'

In [ ]:
sorted_stats = {k: v for k, v in sorted(stats_dictionary.items(), key=lambda item: item[0], reverse=True)}

In [ ]:
sorted_stats

## NIFTY

In [56]:
# signals_df2.tail(130)

In [35]:
bnf2 = resample(pl.DataFrame(bnf1), "30m", "15m")
data = linear_regression_vectorized(bnf2.to_pandas(), 20, 10)
# print(data.tail(10).to_string())
signals_df1 = generate_signals1(data, 4)
signals_df1 = signals_df1.rename(columns={"open": "o", "high": "h", "low": "l", "close": "c"})
new_tb = execute(signals_df1, 4)

# new_tb = execute(signals_df1)
new_tb["DD%"] = new_tb["ROI%"].cumsum() - new_tb["ROI%"].cumsum().cummax()
tradebook_buy_side = new_tb
# new_tb.tail(25)

In [36]:
# tradebook_buy_side['DD%'] = (tradebook_buy_side['ROI%'].cumsum() - tradebook_buy_side['ROI%'].cumsum().cummax())
# tradebook_buy_side

In [37]:
stats2 = generate_stats(new_tb, "...")
roi_overall, stats_overall = next(iter(stats2.items()))
stats_overall

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2017,50.9849,149,40.9396,2.7176,-1.3044,-11.7579,4.3362,...
2018,38.8996,139,41.7266,3.3688,-1.9320,-28.4217,1.3687,...
2019,72.9224,122,43.4426,4.1850,-2.1577,-23.1758,3.1465,...
2020,156.3541,151,43.0464,6.6275,-3.1911,-34.6281,4.5152,...
2021,114.7110,138,48.5507,4.3187,-2.4598,-23.3700,4.9085,...
2022,65.3621,139,38.8489,4.8798,-2.3311,-31.9077,2.0485,...
2023,81.4258,135,42.9630,3.7075,-1.7352,-24.2438,3.3586,...
2024,19.6732,53,41.5094,4.0667,-2.2514,-17.8705,1.1009,...
Overall,600.3332,1026,42.6901,4.2720,-2.1612,-34.6281,17.3366,...


# Short Side Simulation Below

In [192]:
stats_dictionary = {}

tf_list = ["3m", "5m", "15m", "30m"]
previous_n_number_of_candles_for_sl = [1, 2, 3, 4]

for i in range(10, 51, 5):
    for j in range(10, 31, 5):
        for k in range(1, 6):
            for n in previous_n_number_of_candles_for_sl:
                for tf in tf_list:
                    if tf == "10m":
                        offset = "5m"
                    elif tf == "30m" or tf == "60m":
                        offset = "15m"
                    else:
                        offset = "0m"
                    variation = f"LR Period: {i} , Trailing LR : {j} , No. of Candles : {k}, TF : {tf} , {n} Candles SL"
                    print(variation)
                    bnf2 = resample(pl.DataFrame(bnf1), tf, offset)
                    data = linear_regression_vectorized(bnf2.to_pandas(), i, j)
                    signals_df2 = generate_signals2(data, k)
                    signals_df2 = signals_df2.rename(columns={"open": "o", "high": "h", "low": "l", "close": "c"})
                    new_tb = execute_short(signals_df2, n)
                    new_tb["DD%"] = new_tb["ROI%"].cumsum() - new_tb["ROI%"].cumsum().cummax()
                    tradebook_sell_side = new_tb
                    stats = generate_stats(tradebook_sell_side, variation)
                    for overall_roi_dd_ratio, stats_df in stats.items():
                        if overall_roi_dd_ratio is not None and overall_roi_dd_ratio > 3:
                            print(stats_df.to_string())
                            stats_dictionary[overall_roi_dd_ratio] = stats_df

LR Period: 10 , Trailing LR : 10 , No. of Candles : 1, TF : 3m , 1 Candles SL
LR Period: 10 , Trailing LR : 10 , No. of Candles : 1, TF : 5m , 1 Candles SL
LR Period: 10 , Trailing LR : 10 , No. of Candles : 1, TF : 15m , 1 Candles SL
LR Period: 10 , Trailing LR : 10 , No. of Candles : 1, TF : 30m , 1 Candles SL
LR Period: 10 , Trailing LR : 10 , No. of Candles : 1, TF : 3m , 2 Candles SL
LR Period: 10 , Trailing LR : 10 , No. of Candles : 1, TF : 5m , 2 Candles SL
LR Period: 10 , Trailing LR : 10 , No. of Candles : 1, TF : 15m , 2 Candles SL
LR Period: 10 , Trailing LR : 10 , No. of Candles : 1, TF : 30m , 2 Candles SL
LR Period: 10 , Trailing LR : 10 , No. of Candles : 1, TF : 3m , 3 Candles SL
LR Period: 10 , Trailing LR : 10 , No. of Candles : 1, TF : 5m , 3 Candles SL
LR Period: 10 , Trailing LR : 10 , No. of Candles : 1, TF : 15m , 3 Candles SL
LR Period: 10 , Trailing LR : 10 , No. of Candles : 1, TF : 30m , 3 Candles SL
LR Period: 10 , Trailing LR : 10 , No. of Candles : 1, TF 

KeyboardInterrupt: 

In [ ]:
sorted_stats = {k: v for k, v in sorted(stats_dictionary.items(), key=lambda item: item[0], reverse=True)}
sorted_stats

In [185]:
bnf2 = resample(pl.DataFrame(bnf1), "30m", "15m")
data = linear_regression_vectorized(bnf2.to_pandas(), 20, 10)
# print(data.tail(10).to_string())
signals_df2 = generate_signals2(data, 2)
signals_df2 = signals_df2.rename(columns={"open": "o", "high": "h", "low": "l", "close": "c"})
new_tb = execute_short(signals_df2, 4)

# new_tb = execute(signals_df1)
new_tb["DD%"] = new_tb["ROI%"].cumsum() - new_tb["ROI%"].cumsum().cummax()
tradebook_sell_side = new_tb
# new_tb.tail(25)

In [186]:
# stats2 = generate_stats(tradebook_sell_side, "...")
# roi_overall, stats_overall = next(iter(stats2.items()))
# stats_overall

In [187]:
tradebook_sell_side.tail(73)

,Signal Generated At,Trade Type,Entry Time,Entry Price,Initial SL,Final SL,Exit Time,Exit Price,Points Captured,After Costs,PnL,Remarks,Qty,Leverage,ROI%,Trade Year,Trade Month,DD%
1082,2024-01-02 09:15:00,SHORT,2024-01-02 09:45:00,21699.0000,21834.3500,21558.1891,2024-01-03 11:15:00,21574.1500,124.8500,120.5227,334450.4509,TSL Hit,2775,6,3.3445,2024,1,-236.1104
1083,2024-01-03 13:45:00,SHORT,2024-01-03 14:15:00,21561.0000,21591.4500,21553.6755,2024-01-04 09:15:00,21605.8000,-44.8000,-49.1167,-136298.7870,Gap Open Outside ISL,2775,6,-1.3630,2024,1,-237.4734
1084,2024-01-05 12:15:00,SHORT,2024-01-05 12:45:00,21685.5500,21744.0500,21744.0500,2024-01-08 09:15:00,21747.6000,-62.0500,-66.3933,-184241.4491,Gap Open Outside ISL,2775,6,-1.8424,2024,1,-239.3158
1085,2024-01-08 11:15:00,SHORT,2024-01-08 11:45:00,21605.3000,21763.9500,21540.7236,2024-01-10 09:15:00,21558.2500,47.0500,42.7336,118585.8649,TSL Hit,2775,6,1.1859,2024,1,-238.1300
1086,2024-01-10 10:15:00,SHORT,2024-01-10 10:45:00,21523.4000,21661.9000,21661.9000,2024-01-11 09:15:00,21688.0000,-164.6000,-168.9211,-472979.1920,Gap Open Outside ISL,2800,6,-4.7298,2024,1,-242.8598
1087,2024-01-16 11:45:00,SHORT,2024-01-16 12:15:00,22062.2000,22124.1500,21636.0727,2024-01-17 11:45:00,21675.4500,386.7500,382.3762,1041975.2404,TSL Hit,2725,6,10.4198,2024,1,-232.4400
1088,2024-01-17 12:15:00,SHORT,2024-01-17 12:45:00,21642.8000,21836.9000,21419.9591,2024-01-18 10:15:00,21452.7000,190.1000,185.7904,515568.4987,TSL Hit,2775,6,5.1557,2024,1,-227.2843
1089,2024-01-18 13:45:00,SHORT,2024-01-18 14:15:00,21434.7500,21539.4000,21539.4000,2024-01-19 09:15:00,21615.2000,-180.4500,-184.7550,-517313.9860,Gap Open Outside ISL,2800,6,-5.1731,2024,1,-232.4575
1090,2024-01-20 11:15:00,SHORT,2024-01-20 11:45:00,21643.5500,21720.3000,21644.3991,2024-01-20 12:45:00,21653.2500,-9.7000,-14.0297,-38932.3620,TSL Hit,2775,6,-0.3893,2024,1,-232.8468
1091,2024-01-20 14:15:00,SHORT,2024-01-20 14:45:00,21586.9000,21663.3500,21663.3500,2024-01-23 09:15:00,21716.7000,-129.8000,-134.1304,-372211.7490,Gap Open Outside ISL,2775,6,-3.7221,2024,1,-236.5689


In [ ]:
import matplotlib.pyplot as plt

# tradebook = both_indices_combined_tb
tradebook = pd.read_csv('MIDFAIR_JJ.csv')
# tradebook.tail(1)

In [ ]:
tradebook["Cumulative Returns"] = tradebook["ROI%"].cumsum()
plt.figure(figsize=(22, 6))
plt.plot(tradebook["Cumulative Returns"], linestyle="-")
plt.title("Combined 4 Indices ROI Over Time")
plt.xlabel("Trade Number")
plt.ylabel("ROI")
plt.grid(True)
plt.show()

In [ ]:
# tradebook